In [ ]:
import os
import nibabel as nib
import numpy as np
from PIL import Image

def save_slices_as_png(data, output_folder, prefix, axis="axial"):
    # Crear la carpeta específica para el eje
    axis_folder = os.path.join(output_folder, axis)
    os.makedirs(axis_folder, exist_ok=True)
    
    if axis == "axial":  # Eje Z
        slices = range(data.shape[2])
        get_slice = lambda i: np.squeeze(data[:, :, i])
    elif axis == "coronal":  # Eje Y
        slices = range(data.shape[1])
        get_slice = lambda i: np.squeeze(data[:, i, :])
    elif axis == "sagittal":  # Eje X
        slices = range(data.shape[0])
        get_slice = lambda i: np.squeeze(data[i, :, :])
    else:
        raise ValueError("Eje inválido. Usa 'axial', 'coronal' o 'sagittal'.")

    saved_count = 0
    for i in slices:
        slice_2d = get_slice(i)

        # Normalizar intensidades al rango 0-255
        slice_normalized = ((slice_2d - np.min(slice_2d)) / 
                            (np.max(slice_2d) - np.min(slice_2d)) * 255).astype(np.uint8)

        # Guardar como imagen PNG
        output_path = os.path.join(axis_folder, f"{prefix}_{axis}_slice_{i:03d}.png")
        Image.fromarray(slice_normalized).save(output_path)

        # Verificar tamaño del archivo y eliminar si es menor a 10 KB
        if os.path.getsize(output_path) <= 10 * 1024:  # 10 KB
            os.remove(output_path)
        else:
            saved_count += 1
    
    if saved_count == 0:
        print(f"No se guardaron imágenes útiles para {prefix} en el eje {axis}.")

def process_individual_nifti(base_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    # Recorrer directorio para encontrar archivos NIfTI
    for root, _, files in os.walk(base_dir):
        for file in files:
            if file.endswith(".nii") or file.endswith(".nii.gz"):
                file_path = os.path.join(root, file)
                print(f"Procesando archivo: {file_path}")
                
                # Identificar el paciente y el tipo de imagen
                file_name = os.path.basename(file).split('.')[0]
                patient_id, image_type = file_name.split('_')[0:2]

                # Cargar archivo NIfTI
                nifti_img = nib.load(file_path)
                data = nifti_img.get_fdata()

                # Verificar dimensiones
                if len(data.shape) != 3:
                    print(f"Advertencia: El archivo {file_path} no es tridimensional.")
                    continue

                # Crear carpetas organizadas por paciente y tipo
                patient_folder = os.path.join(output_dir, patient_id)
                type_output_folder = os.path.join(patient_folder, image_type)
                os.makedirs(type_output_folder, exist_ok=True)

                # Guardar en los ejes axial, coronal y sagital
                for axis in ["axial", "coronal", "sagittal"]:
                    save_slices_as_png(data, type_output_folder, file_name, axis)

# Directorios
base_dir = "C:/Users/alvar/Downloads/PKG - UPENN-GBM-NIfTI/UPENN-GBM/NIfTI-files/images_structural"  
output_dir = "C:/Users/alvar/Downloads/PKG - UPENN-GBM-NIfTI/UPENN-GBM/NIfTI-files/Imagenes_clasificadas_png"    

process_individual_nifti(base_dir, output_dir)

In [ ]:
#Las imágenes estarán organizadas en función de ID del paciente, a continuación 11 (primera resonancia)/ 21 (Segunda resonancia), corte (axial, sagital, coronal y finalmente las imágenes de los 4 tipos de contraste.

In [ ]:
import shutil

def reorganize_images_by_filename(input_dir, output_dir):
   
    os.makedirs(output_dir, exist_ok=True)

    for root, _, files in os.walk(input_dir):
        for file in files:
            if file.endswith(".png"):
                # Descomponer el nombre del archivo
                # Ejemplo: UPENN-GBM-00630_21_T2_sagittal_slice_093.png
                parts = file.split("_")
                if len(parts) < 5:
                    print(f"Archivo ignorado por formato inesperado: {file}")
                    continue
                
                # Extraer información del nombre del archivo
                patient_id = f"{parts[0]}_{parts[1]}"  # UPENN-GBM-00630
                image_type = parts[2]  # T2
                axis = parts[3]  # sagittal

                # Crear las carpetas organizadas
                patient_folder = os.path.join(output_dir, patient_id)
                type_folder = os.path.join(patient_folder, image_type)
                axis_folder = os.path.join(type_folder, axis)
                os.makedirs(axis_folder, exist_ok=True)

                # Copiar el archivo al nuevo destino
                src_path = os.path.join(root, file)
                dest_path = os.path.join(axis_folder, file)
                shutil.copy(src_path, dest_path)
                print(f"Imagen copiada: {src_path} -> {dest_path}")

    print("Reorganización completada.")

# Directorios
input_dir = "C:/Users/alvar/Downloads/PKG - UPENN-GBM-NIfTI/UPENN-GBM/NIfTI-files/Imagenes_clasificadas_png"  # Carpeta donde están las imágenes actuales
output_dir = "C:/Users/alvar/Downloads/PKG - UPENN-GBM-NIfTI/UPENN-GBM/NIfTI-files/Imagenes_clasificadas_2"  # Carpeta destino para las imágenes organizadas

reorganize_images_by_filename(input_dir, output_dir)


In [ ]:
#La nueva carpeta tendrá las imágenes organizadas en carpetas en función de ID del paciente, contraste, corte y finalmente las imágenes de todos los pacientes que corresponden con el corte y contraste aplicado.

In [ ]:
def reorganize_images_by_filename_v2(input_dir, output_dir):
    """
    Reorganiza las imágenes en carpetas basándose en su nombre, copiando las imágenes 
    sin reemplazar las existentes.

    Args:
        input_dir (str): Carpeta donde están las imágenes generadas.
        output_dir (str): Carpeta donde se almacenarán las imágenes reorganizadas.
    """
    os.makedirs(output_dir, exist_ok=True)

    for root, _, files in os.walk(input_dir):
        for file in files:
            if file.endswith(".png"):
                # Descomponer el nombre del archivo
                # Ejemplo: UPENN-GBM-00630_21_T2_sagittal_slice_093.png
                parts = file.split("_")
                if len(parts) < 5:
                    print(f"Archivo ignorado por formato inesperado: {file}")
                    continue
                
                # Extraer información del nombre del archivo
                patient_id = f"{parts[0]}_{parts[1]}"  # UPENN-GBM-00630
                image_type = parts[2]  # T2
                axis = parts[3]  # sagittal

                # Crear las carpetas organizadas
                patient_folder = os.path.join(output_dir, image_type)
                type_folder = os.path.join(patient_folder, axis)
                os.makedirs(type_folder, exist_ok=True)

                # Copiar el archivo al nuevo destino sin reemplazar los existentes
                src_path = os.path.join(root, file)
                dest_path = os.path.join(type_folder, file)

                # Si la imagen ya existe en el destino, no hacer nada
                if not os.path.exists(dest_path):
                    shutil.copy(src_path, dest_path)
                    print(f"Imagen copiada: {src_path} -> {dest_path}")
                else:
                    print(f"Imagen ya existe, no se ha copiado: {dest_path}")

    print("Reorganización completada.")

# Directorios
input_dir = "C:/Users/alvar/Downloads/PKG - UPENN-GBM-NIfTI/UPENN-GBM/NIfTI-files/Imagenes_clasificadas_2"  # Carpeta donde están las imágenes actuales
output_dir = "C:/Users/alvar/Desktop/Bioinfor y Bioest/TFM\Base de datos Glioblastoma UPENN/Imagenes_clasificadas_3"  # Carpeta destino para la reorganización

# Ejecutar la reorganización
reorganize_images_by_filename_v2(input_dir, output_dir)

In [ ]:
#Las imágenes quedarán organizadas en función del tipo de contraste, después el corte y finalmente las imágenes para cada paciente correspondientes al contraste y corte especificado.